[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/python35/IINTS-SDK/blob/main/examples/notebooks/01_Presets_and_Scenarios.ipynb)

# Presets and Scenario Validation

**Goal:** discover presets and validate your own scenario input.


In [1]:
from __future__ import annotations
from pathlib import Path
import os
import sys
import subprocess


def _find_repo_root() -> Path | None:
    for root in [Path.cwd(), *Path.cwd().parents]:
        if (root / "pyproject.toml").exists() and (root / "src").exists():
            return root
    return None

repo_root = _find_repo_root()
if repo_root is None:
    try:
        import google.colab  # type: ignore
        in_colab = True
    except Exception:
        in_colab = False

    if not in_colab:
        raise RuntimeError("Run this notebook inside the IINTS-SDK repo or on Colab.")

    if not Path("IINTS-SDK").exists():
        subprocess.check_call(["git", "clone", "https://github.com/python35/IINTS-SDK.git"])
    repo_root = Path("IINTS-SDK").resolve()

os.chdir(repo_root)
sys.path.insert(0, str(repo_root / "src"))
print("Repo root:", repo_root)


Repo root: /home/runner/work/IINTS-SDK/IINTS-SDK


## Step 1: List available presets


In [2]:
from iints.presets import load_presets, get_preset

presets = load_presets()
[(p["name"], p.get("description")) for p in presets]


[('baseline_t1d',
  'Clinic-safe baseline with stable parameters and moderate meals.'),
 ('stress_test_meal',
  'Clinic-safe meal stress test with larger lunch and safety clamps enabled.'),
 ('hypo_prone_night',
  'Overnight hypo risk assessment with reduced basal and conservative carbs.'),
 ('hyper_challenge',
  'Post-prandial hyperglycemia challenge with a large meal.'),
 ('pizza_paradox',
  'Delayed glucose rise after a high-fat meal (pizza paradox).'),
 ('midnight_crash', 'Overnight crash risk after evening exercise.')]

## Step 2: Inspect one preset


In [3]:
preset = get_preset("baseline_t1d")
preset["scenario"]


{'scenario_name': 'Clinic Safe Baseline',
 'scenario_version': '1.0',
 'stress_events': [{'start_time': 60,
   'event_type': 'meal',
   'value': 45,
   'absorption_delay_minutes': 15,
   'duration': 60},
  {'start_time': 360,
   'event_type': 'meal',
   'value': 60,
   'absorption_delay_minutes': 20,
   'duration': 90},
  {'start_time': 720,
   'event_type': 'meal',
   'value': 70,
   'absorption_delay_minutes': 15,
   'duration': 90},
  {'start_time': 1000,
   'event_type': 'exercise',
   'value': 0.4,
   'duration': 45}]}

## Step 3: Validate a custom scenario


In [4]:
from iints.validation import validate_scenario_dict, scenario_to_payloads

custom_scenario = {
    "scenario_name": "Custom Lunch",
    "scenario_version": "1.0",
    "stress_events": [
        {"start_time": 60, "event_type": "meal", "value": 50, "absorption_delay_minutes": 10, "duration": 60}
    ],
}

validated = validate_scenario_dict(custom_scenario)
scenario_to_payloads(validated)


[{'start_time': 60,
  'event_type': 'meal',
  'value': 50.0,
  'reported_value': None,
  'absorption_delay_minutes': 10,
  'duration': 60}]

### Recap
You can now build and validate your own scenarios safely.
